# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backbone.utils.wfo_utils import run_strategy, run_wfo
from backbone.utils.general_purpose import calculate_units_size, diff_pips

import itertools
import numpy as np
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta

random.seed(42)

In [3]:
INITIAL_CASH = 10_000
MARGIN = 1/30
COMMISSION = 7e-4

timezone = pytz.timezone("Etc/UTC")
date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

#
limited_testing_start_date = Timestamp('2022-01-01 00:00:00', tz='UTC')
limited_testing_end_date = Timestamp('2024-09-01 00:00:00', tz='UTC')

In [4]:
groups = [
    'Forex',
    'Cash CFD',
    'Cash II CFD',
    'Metals CFD',
    'Crypto CFD',
    'Exotics',
    'Equities I CFD',
    'Equities II CFD',
    'Agriculture',
    'Commodities'
]

In [5]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

symbols = mt5.symbols_get()

tickers = [symbol.path.split('\\')[1] for symbol in symbols if (
    ('Agriculture' in symbol.path)
    or ('Cash CFD' in symbol.path)
    or ('Cash II CFD' in symbol.path)
    or ('Crypto CFD' in symbol.path)
    or ('Equities I CFD' in symbol.path)
    or ('Equities II CFD' in symbol.path)
    or ('Agriculture' in symbol.path)
    or ('Equities II CFD' in symbol.path)
    or ('Commodities' in symbol.path)
    # or ('Forex' in symbol.path)
    )
]

print(tickers)

intervals = [
    mt5.TIMEFRAME_H6,
    mt5.TIMEFRAME_H4,
    mt5.TIMEFRAME_H3,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_M15,
]

symbols = get_data(tickers, intervals, date_from_get_data, date_to_get_data)

['GER40.cash', 'UK100.cash', 'US100.cash', 'US30.cash', 'US500.cash', 'AUS200.cash', 'EU50.cash', 'FRA40.cash', 'HK50.cash', 'JP225.cash', 'N25.cash', 'SPN35.cash', 'US2000.cash', 'UKOIL.cash', 'USOIL.cash', 'ADAUSD', 'BTCUSD', 'DASHUSD', 'DOGEUSD', 'DOTUSD', 'ETHUSD', 'LTCUSD', 'NEOUSD', 'XMRUSD', 'XRPUSD', 'AAPL', 'AMZN', 'BABA', 'BAC', 'GOOG', 'MSFT', 'NFLX', 'NVDA', 'PFE', 'RACE', 'T', 'TSLA', 'V', 'WMT', 'ZM', 'AIRF', 'ALVG', 'BAYGn', 'DBKGn', 'IBE', 'LVMH', 'VOWG_p', 'META', 'COCOA.c', 'COFFEE.c', 'CORN.c', 'SOYBEAN.c', 'WHEAT.c', 'NATGAS.cash']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
GER40.cash
GER40.cash
GER40.cash
GER40.cash
GER40.cash
UK100.cash
UK100.cash
UK100.cash
UK100.cash
UK100.cash
US100.cash
US100.cash
US100.cash
US100.cash
US100.cash
US30.cash
US30.cash
US30.cash
US30.cash
US30.cash
US500.cash
US500.cash
US500.cash
US500.cash
US500.cash
AUS200.cash
AUS200.cash
AUS200.cash
AUS200.cash
AUS200.cash
EU50.cash
EU50.cash
EU50.cas

In [6]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates

print(intervals_start_dates)
print(limited_testing_start_date)
print(limited_testing_end_date)

{16390: Timestamp('2024-10-23 00:00:00+0000', tz='UTC'), 16388: Timestamp('2024-10-23 00:00:00+0000', tz='UTC'), 16387: Timestamp('2024-10-23 00:00:00+0000', tz='UTC'), 16386: Timestamp('2024-10-23 00:00:00+0000', tz='UTC'), 16385: Timestamp('2024-10-23 01:00:00+0000', tz='UTC')}
2022-01-01 00:00:00+00:00
2024-09-01 00:00:00+00:00


# Estrategia

In [7]:
from backbone.b_percent_strategy import BPercent
   
STRATEGY = BPercent

# Analisis preliminar

In [8]:
strategies = [
    STRATEGY
]

experiments = list(itertools.product(
    tickers, intervals, strategies
))

stats_per_symbol = {}

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    try:
        print(ticker, interval)
        
        if ticker not in stats_per_symbol.keys():
            stats_per_symbol[ticker] = {}
        
        prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

        df_stats, stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=prices, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN
        )

        performance = pd.concat([performance, df_stats])
        stats_per_symbol[ticker][interval] = stats

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


AAPL 16390
AAPL 16388
AAPL 16387
AAPL 16386
AAPL 16385
ADAUSD 16390
ADAUSD 16388
ADAUSD 16387
ADAUSD 16386
ADAUSD 16385
AIRF 16390
AIRF 16388
AIRF 16387
AIRF 16386
AIRF 16385
ALVG 16390
ALVG 16388
ALVG 16387
ALVG 16386
ALVG 16385
AMZN 16390
AMZN 16388
AMZN 16387
AMZN 16386
AMZN 16385
AUS200.cash 16390
AUS200.cash 16388
AUS200.cash 16387
AUS200.cash 16386
AUS200.cash 16385
BABA 16390
BABA 16388
BABA 16387
BABA 16386
BABA 16385
BAC 16390
BAC 16388
BAC 16387
BAC 16386
BAC 16385
BAYGn 16390
BAYGn 16388
BAYGn 16387
BAYGn 16386
BAYGn 16385
BTCUSD 16390
BTCUSD 16388
BTCUSD 16387
BTCUSD 16386
BTCUSD 16385
COCOA.c 16390
COCOA.c 16388


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


COCOA.c 16387
COCOA.c 16386


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


COCOA.c 16385


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


COFFEE.c 16390
COFFEE.c 16388
COFFEE.c 16387
COFFEE.c 16386
COFFEE.c 16385
CORN.c 16390
CORN.c 16388
CORN.c 16387
CORN.c 16386
CORN.c 16385
DASHUSD 16390
DASHUSD 16388
DASHUSD 16387
DASHUSD 16386
DASHUSD 16385
DBKGn 16390
DBKGn 16388
DBKGn 16387
DBKGn 16386
DBKGn 16385
DOGEUSD 16390
DOGEUSD 16388
DOGEUSD 16387
DOGEUSD 16386
DOGEUSD 16385
DOTUSD 16390
DOTUSD 16388
DOTUSD 16387
DOTUSD 16386
DOTUSD 16385
ETHUSD 16390
ETHUSD 16388
ETHUSD 16387
ETHUSD 16386
ETHUSD 16385
EU50.cash 16390
EU50.cash 16388
EU50.cash 16387
EU50.cash 16386
EU50.cash 16385
FRA40.cash 16390
FRA40.cash 16388
FRA40.cash 16387
FRA40.cash 16386
FRA40.cash 16385
GER40.cash 16390
GER40.cash 16388
GER40.cash 16387
GER40.cash 16386
GER40.cash 16385
GOOG 16390
GOOG 16388
GOOG 16387
GOOG 16386
GOOG 16385
HK50.cash 16390
HK50.cash 16388
HK50.cash 16387
HK50.cash 16386
HK50.cash 16385
IBE 16390
IBE 16388
IBE 16387
IBE 16386
IBE 16385
JP225.cash 16390
JP225.cash 16388
JP225.cash 16387
JP225.cash 16386
JP225.cash 16385
LTCUSD 163

In [27]:

filter_performance = performance[
    (performance['return/dd']>1)
    # & (performance['stability_ratio']>0.7)
].sort_values(by=['custom_metric'], ascending=[False])#.drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BPercent,SOYBEAN.c,16386,0.318656,9.853075,10985.307540,7.837561,176 days 04:00:00,47.368421,0.913313,38,0.223152,16.647961,10985.307540,392 days 04:00:00,1.257161,4.084537
0,BPercent,WMT,16390,0.443658,7.007744,10700.774399,6.397907,321 days 00:00:00,56.000000,0.493681,25,0.438622,18.144611,10700.774399,1064 days 06:00:00,1.095318,3.086266


In [25]:
filter_performance = performance[performance['ticker'].isin(portfolio)].sort_values(by=['ticker', 'interval'], ascending=[False, True])[
    [
        'strategy',
        'ticker',
        'interval',
        'stability_ratio',
        'trades',
        'return',
        'drawdown',
        'return/dd',
        'custom_metric'
        ]
]#.drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance.tail(50))

pd.DataFrame(filter_performance.groupby(['ticker']).agg({'return/dd':['min', 'max', 'mean','std', 'median']}).sort_values(by=('return/dd', 'mean')))



,strategy,ticker,interval,stability_ratio,trades,return,drawdown,return/dd,custom_metric
0,BPercent,WMT,16386,0.039874,35,-7.358145,10.166427,-0.723769,-2.361369
0,BPercent,WMT,16387,0.365528,30,0.417916,8.885810,0.047032,0.145170
0,BPercent,WMT,16388,0.136364,26,1.266254,7.653312,0.165452,0.482285
0,BPercent,WMT,16390,0.443658,25,7.007744,6.397907,1.095318,3.086266
0,BPercent,SOYBEAN.c,16386,0.318656,38,9.853075,7.837561,1.257161,4.084537
0,BPercent,SOYBEAN.c,16388,0.019591,16,0.204224,6.135224,0.033287,0.081092


return/dd                                        
                min       max      mean       std    median
ticker                                                     
WMT       -0.723769  1.095318  0.146008  0.745626  0.106242
SOYBEAN.c  0.033287  1.257161  0.645224  0.865409  0.645224

In [29]:
filter_performance = performance[
    ((performance['ticker'] == 'XMRUSD')
    | (performance['ticker'] == 'WMT')
    | (performance['ticker'] == 'AIRF')
    | (performance['ticker'] == 'NEOUSD')
    | (performance['ticker'] == 'SOYBEAN.c'))
].sort_values(by=['ticker'], ascending=[False])

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

filter_performance

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BPercent,XMRUSD,16386,0.631301,5.876106,10587.610637,12.691320,549 days 16:00:00,40.875912,0.208025,137,-0.232559,22.670065,10587.610637,1066 days 00:00:00,0.463002,2.114702
0,BPercent,XMRUSD,16387,0.420665,0.934785,10093.478488,10.979214,501 days 12:00:00,41.237113,0.045252,97,-0.147481,19.200095,10093.478488,1066 days 00:00:00,0.085141,0.357783
0,BPercent,XMRUSD,16388,0.000258,-7.288845,9271.115546,15.187913,984 days 00:00:00,34.426230,0.000000,61,-0.999383,16.624526,9271.115546,1066 days 00:00:00,-0.479911,-1.858303
0,BPercent,XMRUSD,16390,0.538538,-7.481972,9251.802767,10.236020,982 days 18:00:00,32.352941,0.000000,34,-1.652011,15.921457,9251.802767,1066 days 00:00:00,-0.730945,-2.367477
0,BPercent,WMT,16386,0.039874,-7.358145,9264.185522,10.166427,339 days 04:00:00,37.142857,0.000000,35,-0.221524,15.234509,9264.185522,1064 days 06:00:00,-0.723769,-2.361369
0,BPercent,WMT,16387,0.365528,0.417916,10041.791620,8.885810,896 days 00:00:00,43.333333,0.025657,30,-0.068057,17.092069,10041.791620,1064 days 06:00:00,0.047032,0.145170
0,BPercent,WMT,16388,0.136364,1.266254,10126.625407,7.653312,579 days 20:00:00,50.000000,0.104839,26,-0.041354,17.180014,10126.625407,1064 days 04:00:00,0.165452,0.482285
0,BPercent,WMT,16390,0.443658,7.007744,10700.774399,6.397907,321 days 00:00:00,56.000000,0.493681,25,0.438622,18.144611,10700.774399,1064 days 06:00:00,1.095318,3.086266
0,BPercent,SOYBEAN.c,16388,0.019591,0.204224,10020.422413,6.135224,176 days 08:00:00,50.000000,0.039840,16,0.077878,12.593516,10020.422413,392 days 04:00:00,0.033287,0.081092
0,BPercent,SOYBEAN.c,16386,0.318656,9.853075,10985.307540,7.837561,176 days 04:00:00,47.368421,0.913313,38,0.223152,16.647961,10985.307540,392 days 04:00:00,1.257161,4.084537


In [12]:
for ticker, interval in zip(portfolio, intervals):
    
    prices = symbols[ticker][interval]

    df_stats = run_strategy(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        commission=COMMISSION, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN,
        plot=True
    )
    
    break

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


# WFO

In [13]:
lookback_bars_per_interval = {
    15: 2000,
    16385: 2000,
    16386: 1800,
    16387: 1800,
    16388: 1200,
    16390: 1200,
    16392: 1200,
}

In [14]:
0/0

ZeroDivisionError: division by zero

In [16]:
from backbone.utils.wfo_utils import optimization_function

validation_bars = 200
warmup_bars = 200

all_wfo_performances = pd.DataFrame()
all_opt_params = {}

params = {
    'b_open_threshold' : [0.9, 0.95, 1, 1.5],
    'b_close_threshold': [0.2, 0.5, 0.6, 0.8, 0.9, 1],
}

for index, row in filter_performance.iterrows():
    
    try:
        ticker = row.ticker
        interval = row.interval
        
        print(ticker, interval)
        
        lookback_bars = lookback_bars_per_interval[interval]

        prices = symbols[ticker][interval]

        wfo_stats, df_stats, opt_params = run_wfo(
            strategy=STRATEGY,
            ticker=ticker,
            interval=interval,
            prices=prices,
            initial_cash=INITIAL_CASH,
            commission=COMMISSION,
            margin=MARGIN,
            optim_func=optimization_function,
            params=params,
            lookback_bars=lookback_bars,
            warmup_bars=warmup_bars,
            validation_bars=validation_bars,
            plot=False
        )
                
        if ticker not in all_opt_params.keys():
            all_opt_params[ticker] = {}

        all_opt_params[ticker][interval] = opt_params

        all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
    
    except:
        print(f'No se pudo ejecutar para el ticker {ticker}')
    
all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

XMRUSD 16386
XMRUSD 16387
XMRUSD 16388
XMRUSD 16390
WMT 16386
WMT 16387
WMT 16388


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: invalid value encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


WMT 16390
SOYBEAN.c 16386
SOYBEAN.c 16387
No se pudo ejecutar para el ticker SOYBEAN.c
SOYBEAN.c 16388
SOYBEAN.c 16390
No se pudo ejecutar para el ticker SOYBEAN.c
NEOUSD 16386
NEOUSD 16387
NEOUSD 16388
NEOUSD 16390
AIRF 16388
AIRF 16386
AIRF 16387
AIRF 16390


In [17]:

performance = pd.DataFrame()

wfo_stats_per_symbol = {}

for index, row in filter_performance.iterrows():
    try:
        ticker = row.ticker
        interval = row.interval
        print(ticker, interval)
        
        prices = symbols[ticker][interval]
        params = all_opt_params[ticker][interval]
        
        if ticker not in wfo_stats_per_symbol.keys():
            wfo_stats_per_symbol[ticker] = {}

        df_stats, wfo_stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=prices, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN, 
            opt_params=params, 
            plot=True
        )

        wfo_stats_per_symbol[ticker][interval] = wfo_stats

        performance = pd.concat([performance, df_stats])

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


XMRUSD 16386


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


XMRUSD 16387


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


XMRUSD 16388


...

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


AIRF 16387


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


AIRF 16390


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


In [20]:
filtered_wfo_performance = performance.sort_values(by='return/dd', ascending=False)

filtered_wfo_performance

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BPercent,SOYBEAN.c,16386,0.318656,9.853075,10985.307540,7.837561,176 days 04:00:00,47.368421,0.913313,38,0.223152,16.647961,10985.307540,392 days 04:00:00,1.257161,4.084537
0,BPercent,WMT,16390,0.443658,7.007744,10700.774399,6.397907,321 days 00:00:00,56.000000,0.493681,25,0.438622,18.144611,10700.774399,1064 days 06:00:00,1.095318,3.086266
0,BPercent,NEOUSD,16386,0.643606,7.588506,10758.850624,9.300777,362 days 00:00:00,38.983051,0.280041,118,0.064206,17.524295,10758.850624,1066 days 00:00:00,0.815900,3.520745
0,BPercent,NEOUSD,16387,0.283659,6.218268,10621.826847,12.833143,546 days 12:00:00,38.235294,0.251008,102,0.133608,16.450423,10621.826847,1066 days 00:00:00,0.484548,2.083401
0,BPercent,XMRUSD,16386,0.631301,5.876106,10587.610637,12.691320,549 days 16:00:00,40.875912,0.208025,137,-0.232559,22.670065,10587.610637,1066 days 00:00:00,0.463002,2.114702
0,BPercent,AIRF,16386,0.003503,4.452171,10445.217098,11.649542,913 days 00:00:00,40.816327,0.225049,49,-0.056664,18.333333,10445.217098,1064 days 08:00:00,0.382176,1.376887
0,BPercent,WMT,16388,0.136364,1.266254,10126.625407,7.653312,579 days 20:00:00,50.000000,0.104839,26,-0.041354,17.180014,10126.625407,1064 days 04:00:00,0.165452,0.482285
0,BPercent,XMRUSD,16387,0.420665,0.934785,10093.478488,10.979214,501 days 12:00:00,41.237113,0.045252,97,-0.147481,19.200095,10093.478488,1066 days 00:00:00,0.085141,0.357783
0,BPercent,WMT,16387,0.365528,0.417916,10041.791620,8.885810,896 days 00:00:00,43.333333,0.025657,30,-0.068057,17.092069,10041.791620,1064 days 06:00:00,0.047032,0.145170
0,BPercent,SOYBEAN.c,16388,0.019591,0.204224,10020.422413,6.135224,176 days 08:00:00,50.000000,0.039840,16,0.077878,12.593516,10020.422413,392 days 04:00:00,0.033287,0.081092


# Montecarlo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from backbone.utils.wfo_utils import montecarlo_statistics_simulation

# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    
    print(f"Procesando ticker: {ticker}")
    trades_history = wfo_stats_per_symbol[ticker][interval]._trades
    eq_curve = wfo_stats_per_symbol[ticker][interval]._equity_curve
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
        equity_curve=eq_curve,
        trade_history=trades_history, 
        n_simulations=100_000, 
        initial_equity=INITIAL_CASH, 
        threshold_ruin=0.8, 
        return_raw_curves=True,
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
    
    montecarlo_simulations[ticker] = mc
    
    synthetic_drawdown_curve['ticker'] = ticker
    synthetic_return_curve['ticker'] = ticker
    
    all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
    all_returns = pd.concat([all_returns, synthetic_return_curve])

In [ ]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Generar el gráfico
plt.figure(figsize=(12, 20))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
